In [1]:
"""
Credits: https://pytorch.org/hub/intelisl_midas_v2/

"""


import cv2
import torch
import urllib.request
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
filename = "edition25_sequence_5.mp4"

DATA_PATH = './lausanne_prix/'
OUTPUT_PATH = './Result_MiDas/'

Load a model (see https://github.com/intel-isl/MiDaS/#Accuracy for an overview)

In [3]:
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in /Users/irinaserenko/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/irinaserenko/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Move model to GPU if available

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
#midas.eval()

In [5]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in /Users/irinaserenko/.cache/torch/hub/intel-isl_MiDaS_master


In [ ]:
video_input = DATA_PATH + filename
video_name = os.path.basename(video_input)

#video_input = './lausanne_prix/edition34_sequence_5.mp4'
video_output = OUTPUT_PATH + video_name
cap = cv2.VideoCapture(video_input)

# Obtain frame size information using get() method
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer object
output = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size, False)

count = 0
with torch.no_grad():
    while cap.isOpened():

        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            #continue
            break

        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        input_batch = transform(img).to(device)


        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
        ).squeeze()

        image = prediction.cpu().numpy()
        
        norm_image = cv2.normalize(image, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

        norm_image = norm_image.astype(np.uint8)

        output.write(norm_image)
        count +=1
        #print(count)

cap.release()
output.release()

### References


@article{Ranftl2020,
	author    = {Ren\'{e} Ranftl and Katrin Lasinger and David Hafner and Konrad Schindler and Vladlen Koltun},
	title     = {Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer},
	journal   = {IEEE Transactions on Pattern Analysis and Machine Intelligence (TPAMI)},
	year      = {2020},
}

@article{Ranftl2021,
	author    = {Ren\'{e} Ranftl and Alexey Bochkovskiy and Vladlen Koltun},
	title     = {Vision Transformers for Dense Prediction},
	journal   = {ArXiv preprint},
	year      = {2021},
}